In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import CuDNNLSTM
from keras.layers import Dense

C:\Users\AdrienMartinez\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sample = 10

def arrayFromDataset(dataset):
    global sample
    size = dataset.shape[0]
    data = dataset.shape[1]
    array = np.zeros((size - sample + 1, sample, data))
    for i in range(size):
        if i >= array.shape[0]:
            break
        array[i] = dataset[i:i+sample]
    return array

def separateDataset(dataset):
    global sample
    sampleSize = int(dataset.shape[0] * 0.8)
    X_train = arrayFromDataset(dataset[:sampleSize, :-3])
    Y_train = dataset[:sampleSize - sample+1, -3:]
    X_test = arrayFromDataset(dataset[sampleSize + 1:, :-3])
    Y_test = dataset[sampleSize + 1: - sample+1, -3:]
    return (X_train, Y_train), (X_test, Y_test)

In [3]:
dataframe = pd.concat([
    pd.read_csv("droite1-simon.csv"),
    pd.read_csv("droite2-simon.csv"),
    pd.read_csv("droite3-simon.csv"),
    pd.read_csv("droite4-simon.csv"),
    pd.read_csv("droite5-simon.csv"),
    pd.read_csv("gauche1-simon.csv"),
    pd.read_csv("gauche2-simon.csv"),
    pd.read_csv("gauche3-simon.csv"),
    pd.read_csv("gauche4-simon.csv"),
    pd.read_csv("gauche5-simon.csv"),
    pd.read_csv("neutre1-simon.csv"),
    pd.read_csv("neutre2-simon.csv"),
    pd.read_csv("neutre3-simon.csv"),
    pd.read_csv("neutre4-simon.csv"),
    pd.read_csv("neutre5-simon.csv")
])

In [4]:
dataframe['left'] = dataframe.metadata.map({"gauche":1, "neutre":0, "droite": 0})
dataframe['right'] = dataframe.metadata.map({"droite":1, "neutre":0, "gauche": 0})
dataframe['neutral'] = dataframe.metadata.map({"droite":0, "neutre":1, "gauche": 0})
dataframe = dataframe.drop(["n", "time", "metadata"], axis=1)
dataframe.head()

,AF3_theta,AF3_alpha,AF3_betaL,AF3_betaH,AF3_gamma,T7_theta,T7_alpha,T7_betaL,T7_betaH,T7_gamma,...,T8_betaH,T8_gamma,AF4_theta,AF4_alpha,AF4_betaL,AF4_betaH,AF4_gamma,left,right,neutral
0,545,324,148,70,96,153,151,161,139,139,...,194,97,555,275,124,128,116,0,1,0
1,697,234,147,63,93,143,138,175,132,142,...,219,98,706,320,144,123,119,0,1,0
2,844,169,157,61,91,142,127,180,129,158,...,224,98,863,370,165,123,120,0,1,0
3,929,139,173,60,91,149,123,172,131,186,...,207,99,949,405,180,123,118,0,1,0
4,934,139,194,60,91,162,126,153,133,220,...,173,99,932,412,188,121,112,0,1,0


In [5]:
dataset = dataframe.values

(X_train, Y_train), (X_test, Y_test) = separateDataset(dataset)

In [6]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1456, 10, 25), (1456, 3), (357, 10, 25), (357, 3))

In [7]:
model = Sequential()
model.add(CuDNNLSTM(32, return_sequences=True, input_shape=X_train.shape[1:]))
model.add(CuDNNLSTM(32))
#model.add(CuDNNLSTM(32))
model.add(Dense(3, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 10, 32)            7552      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 32)                8448      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 16,099
Trainable params: 16,099
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train, Y_train, epochs=300, batch_size=10)

Epoch 1/300
1456/1456 [==============================] - 9s 6ms/step - loss: 0.6415 - acc: 0.7452
Epoch 2/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2981 - acc: 0.8970
Epoch 3/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2297 - acc: 0.9183
Epoch 4/300
1456/1456 [==============================] - 4s 3ms/step - loss: 0.2387 - acc: 0.9080
Epoch 5/300
1456/1456 [==============================] - 4s 3ms/step - loss: 0.1087 - acc: 0.9629
Epoch 6/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0868 - acc: 0.9698
Epoch 7/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0775 - acc: 0.9725
Epoch 8/300
1456/1456 [==============================] - 4s 3ms/step - loss: 0.1145 - acc: 0.9636
Epoch 9/300
1456/1456 [==============================] - 4s 2ms/step - loss: 0.0754 - acc: 0.9753
Epoch 10/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0280 - acc: 0.9924
Epoch 11/300
1456/1

1456/1456 [==============================] - 3s 2ms/step - loss: 0.1840 - acc: 0.9245
Epoch 84/300
1456/1456 [==============================] - 4s 2ms/step - loss: 0.0646 - acc: 0.9760
Epoch 85/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0522 - acc: 0.9815
Epoch 86/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0326 - acc: 0.9876
Epoch 87/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0509 - acc: 0.9842
Epoch 88/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0217 - acc: 0.9959
Epoch 89/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0175 - acc: 0.9959
Epoch 90/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0206 - acc: 0.9945
Epoch 91/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0647 - acc: 0.9815
Epoch 92/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.0305 - acc: 0.9897
Epoch 93/300
1456/1456 

1456/1456 [==============================] - 3s 2ms/step - loss: 0.3724 - acc: 0.8729
Epoch 166/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.4549 - acc: 0.8008
Epoch 167/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.3798 - acc: 0.8420
Epoch 168/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.3460 - acc: 0.8386A:
Epoch 169/300
1456/1456 [==============================] - 4s 3ms/step - loss: 0.3168 - acc: 0.8757
Epoch 170/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2934 - acc: 0.8805
Epoch 171/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2934 - acc: 0.8812
Epoch 172/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2682 - acc: 0.8846
Epoch 173/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2448 - acc: 0.8970
Epoch 174/300
1456/1456 [==============================] - 3s 2ms/step - loss: 0.2513 - acc: 0.8915
Epoch 175/30

1456/1456 [==============================] - 3s 2ms/step - loss: 4.6335e-05 - acc: 1.0000
Epoch 247/300
1456/1456 [==============================] - 3s 2ms/step - loss: 4.3334e-05 - acc: 1.0000
Epoch 248/300
1456/1456 [==============================] - 3s 2ms/step - loss: 3.8657e-05 - acc: 1.0000
Epoch 249/300
1456/1456 [==============================] - 3s 2ms/step - loss: 3.4998e-05 - acc: 1.0000
Epoch 250/300
1456/1456 [==============================] - 3s 2ms/step - loss: 3.2317e-05 - acc: 1.0000
Epoch 251/300
1456/1456 [==============================] - 3s 2ms/step - loss: 2.9670e-05 - acc: 1.0000
Epoch 252/300
1456/1456 [==============================] - 4s 2ms/step - loss: 2.6947e-05 - acc: 1.0000
Epoch 253/300
1456/1456 [==============================] - 3s 2ms/step - loss: 2.4701e-05 - acc: 1.0000
Epoch 254/300
1456/1456 [==============================] - 3s 2ms/step - loss: 2.2654e-05 - acc: 1.0000
Epoch 255/300
1456/1456 [==============================] - 3s 2ms/step - loss:

In [9]:
model.predict(X_test)

array([[0.00715253, 0.3380261 , 0.6548214 ],
       [0.00715253, 0.3380261 , 0.6548214 ],
       [0.00715253, 0.3380261 , 0.6548214 ],
       ...,
       [0.00715365, 0.33804664, 0.65479976],
       [0.00391339, 0.9889535 , 0.00713307],
       [0.005325  , 0.9930756 , 0.00159938]], dtype=float32)

In [10]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

Score : 83.19%
